In [19]:
import os
import sys
from os.path import join
import json

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import nfl_data_py as nfl

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(ROOT_DIR,'py'))

import util
from plot.plot_simple import plot_play_with_speed

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

with open("paths.json", 'r') as f:
    paths = json.load(f)

PROCESSED_DATA_PATH = paths['processed_data']

In [38]:
run_concepts = pd.read_pickle(join(PROCESSED_DATA_PATH, 'run_concepts.pkl'))

motion = pd.DataFrame()
for wk in range(1,10):
    tmp = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{wk}', 'motion_plays.pkl'))
    motion = pd.concat([motion, tmp])
motion = motion.reset_index(drop=True)
motion['motion_group'] = motion.motion_group.fillna('DROP')

df = run_concepts.merge(motion, on='game_play_id', how='left')

del tmp, motion, run_concepts

In [47]:
df = df[(df.motion_group != 'DROP') & (df.motion_group != 'nan')].reset_index(drop=True)

In [48]:
df.head()

,game_play_id,run_concept,play_dir,play_dir_location,motion_nfl_id,motion_position,motion_had_rush_attempt,initial_alignment,n_direction_changes,same_motion_dir,motion_dir_first,motion_dir_last,pre_snap_motion_dist_traveled,motion_group,motion_sub_group
0,2022091105_3712,OUTSIDE ZONE,right,inside-right,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022091104_1094,OUTSIDE ZONE,right,outside-right,47791.0,TE,0.0,None,0.0,right-all,right,right,0.697350,Jet,None
2,2022091101_3923,INSIDE ZONE,right,middle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022091106_1380,INSIDE ZONE,right,inside-right,53435.0,WR,0.0,None,0.0,right-all,right,right,8.234742,Orbit,None
4,2022091100_501,OUTSIDE ZONE,left,outside-left,54476.0,WR,0.0,None,0.0,left-all,left,left,9.052830,Orbit,None


In [49]:
df.isna().sum()

game_play_id                        0
run_concept                         0
play_dir                            0
play_dir_location                   0
motion_nfl_id                    3077
motion_position                  3077
motion_had_rush_attempt          3077
initial_alignment                4510
n_direction_changes              3077
same_motion_dir                  3241
motion_dir_first                 3077
motion_dir_last                  3077
pre_snap_motion_dist_traveled    3077
motion_group                     3077
motion_sub_group                 4257
dtype: int64

In [50]:
df.motion_group.value_counts()

Jet                675
Orbit              337
Split-Flow Over    292
Fly                223
Condense           174
Yo-Yo              136
Name: motion_group, dtype: int64